# Coin 데이터 가져오기

* API 참고 Github

> https://github.com/sharebook-kr/pyupbit

In [2]:
import pyupbit
import requests
import pandas as pd
import numpy as np

## 1. 상장된 모든 코인 List 출력

In [3]:
# 마켓 코드 조회
def coin_list():
    url = "https://api.upbit.com/v1/market/all"
    querystring = {"isDetails":"false"}
    headers = {"Accept": "application/json"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    jsonData = response.json()

    coin_codes = pd.DataFrame(jsonData)
    return coin_codes

In [4]:
want_coins = ['비트코인', '이오스', '도지코인']
# want_coins = ['비트코인']           # test

# 업비트의 코인 리스트 확인
coin_codes = coin_list()
coin_codes

,market,korean_name,english_name
0,KRW-BTC,비트코인,Bitcoin
1,KRW-ETH,이더리움,Ethereum
2,BTC-ETH,이더리움,Ethereum
3,BTC-LTC,라이트코인,Litecoin
4,BTC-XRP,리플,Ripple
...,...,...,...
250,KRW-DAWN,던프로토콜,Dawn Protocol
251,KRW-AXS,엑시인피니티,Axie Infinity
252,KRW-STX,스택스,Stacks
253,BTC-GRT,그래프,The Graph


In [50]:
pyupbit.get_ohlcv('KRW-ETH', interval="minute30", to="2017-09-28")

,open,high,low,close,volume,value
2017-09-25 12:00:00,324500.0,325500.0,321500.0,323500.0,627.317887,2.028068e+08
2017-09-25 12:30:00,323500.0,324500.0,320000.0,322500.0,675.714403,2.178292e+08
2017-09-25 13:00:00,322500.0,324000.0,322000.0,322500.0,118.801837,3.836718e+07
2017-09-25 22:00:00,323000.0,323000.0,322000.0,322000.0,0.000900,2.905500e+02
2017-09-25 22:30:00,322500.0,322500.0,321000.0,321000.0,0.001100,3.539000e+02
...,...,...,...,...,...,...
2017-09-27 21:30:00,327500.0,328500.0,327000.0,327000.0,0.001100,3.607000e+02
2017-09-27 22:00:00,327500.0,327500.0,325500.0,327000.0,0.001100,3.594000e+02
2017-09-27 22:30:00,326000.0,327500.0,326000.0,327000.0,0.001100,3.595000e+02
2017-09-27 23:00:00,327000.0,328500.0,327000.0,328500.0,0.001100,3.605000e+02


## 2. 코인 데이터 가져오기

In [7]:
# 일주일 코인 변동 확인
def load_coin_data(coin, interval, from_=None, to=None, days=None):
    '''
    parm:
        coin: korean name of wanted coin
        interval: time interval (minute[1, 3, 5, 15, 10, 30, 60, 240], week, day, month)
        from_: 원하는 데이터의 시작 날짜
        to: 원하는 데이터의 끝 날짜
        days: 현재부터 n일 전까지의 데이터
    '''
    import datetime
    global coin_codes
    
    # 전체 코인 리스트에서 특정 코인 정보 가져오기
    coin_code = coin_codes[coin_codes.korean_name == coin]
    code = coin_code[coin_code.market.str.contains('KRW')]['market'].iloc[0]

    # datetime 형식으로 변환
    if from_ != None and days != None:
        print("Input Error : from_과 days중 1가지만 입력해 주세요")
        return -1
    elif days == None:
        from_ = pd.to_datetime(from_)
    else:
        from_ = pd.to_datetime(to) - datetime.timedelta(days=days)
    
    
    # 데이터 가져오기
    data_df = pyupbit.get_ohlcv(code, interval=interval, to=to)

    # 입력 오류 처리
    if data_df is None:
        print("Coin Name Error: 이름을 잘못 입력하셨습니다\n잘못된 이름: {}".format(coin))
        return -1
    
    # 원하는 날짜 까지의 데이터 가져오기
    if from_ == None:
        return data_df
    else:
        timeout = 0
        timeout_bool = False
        # 원하는 시작 위치까지 계속 반복(한번에 200개 데이터씩 가능)
        start = data_df.iloc[0].name
        while start > from_:
            additional_df = pyupbit.get_ohlcv(code, interval=interval, to=start)
            if additional_df is None:
                print("서버와의 연결 불안정 재시도중...{}".format(timeout))
                timeout += 1
                if timeout > 100:
                    print("서버와의 연결 실패 or 이전 데이터가 없습니다...")
                    break
                continue
            timeout = 0
            data_df = pd.concat([additional_df, data_df])
            data_df.sort_index(inplace=True)
            start = data_df.iloc[0].name
        else:
            data_df = data_df.loc[from_:]
            return data_df
        return data_df

## 원하는 데이터 정보 입력, 데이터 가져오기
### 입력 값
* 코인의 종류
* 데이터의 시간 간격
* 데이터의 시작 시간
* 데이터의 종료 시간 or n일 전

### 참고
* 2017-09-25 12:00:00 부터 데이터 있음

In [68]:
# 찾고싶은 코인에 대한 일주일 정보 사전
if __name__ == '__main__':
    interval_kinds = ['minute1', 'minute3', 'minute5', 'minute10',
                      'minute15', 'minute30', 'minute60', 'minute240',
                      'week', 'day', 'month']
    
    # 필요 변수 입력
    print("=============원하는 코인 데이터 가져오기=============")
    # want_coins = ['비트 코인', '이오스', '도지코인']  # test
    want_coins = list(input("1. 원하는 코인의 각각의 이름(띄어쓰기 기준 분류)\n입력: ").split())
    interval = input("2. 원하는 시간 간격을 입력해 주세요\n※ 참고 - 시간간격의 종류\n{}\n".format(", ".join(interval_kinds)))
    from_=None
    days=None
    while True:
        want_type = int(input("3. 얻고싶은 데이터의 기간을 선택 2가지 종류중 원하는 종류의 숫자를 입력해 주세요\n1 : 시작시간 ~ 종료시간을 입력\n2 : 특정시간 ~ n일 이전 데이터\n입력: "))
        if want_type == 1:
            from_ = input("코인 데이터의 원하는 시작시간을 입력해 주세요\n※ 참고 - 예: 2020-10-01 04:34:20\n입력: ")
            to = input("코인 데이터의 원하는 종료 시간을 입력해 주세요(입력 안할시 현재 시간)\n※ 참고 - 예: 2021-03-18 18:15:43\n입력: ")
            if to == "":
                to = None
            break
        elif want_type == 2:
            to = input("코인 데이터의 원하는 종료 시간을 입력해 주세요(입력 안할시 현재 시간)\n※ 참고 - 예: 2020-10-01 18:15:43\n입력: ")
            if to == "":
                to = None
            days = int(input("몇일 전까지의 데이터를 원하는지 입력해 주세요\n※ 참고 - 예: 7\n입력: "))
            break
        else:
            print("다시입력 해주세요")
            
    # 최종 데이터 저장 딕셔너리
    coins_week_info = {}

    # 업비트의 코인 리스트 확인
    coin_codes = coin_list()

    # 원하는 데이터 불러와서 저장
    for want_coin in want_coins:
        coins_week_info[want_coin] = load_coin_data(want_coin, interval, from_=from_, to=to, days=days)
        
    print(coins_week_info)

=============원하는 코인 데이터 가져오기=============


1. 원하는 코인의 각각의 이름(띄어쓰기 기준 분류)
입력:  비트코인 이더리움
2. 원하는 시간 간격을 입력해 주세요
※ 참고 - 시간간격의 종류
minute1, minute3, minute5, minute10, minute15, minute30, minute60, minute240, week, day, month
 minute30
3. 얻고싶은 데이터의 기간을 선택 2가지 종류중 원하는 종류의 숫자를 입력해 주세요
1 : 시작시간 ~ 종료시간을 입력
2 : 특정시간 ~ n일 이전 데이터
입력:  1
코인 데이터의 원하는 시작시간을 입력해 주세요
※ 참고 - 예: 2020-10-01 04:34:20
입력:  2017-09-25
코인 데이터의 원하는 종료 시간을 입력해 주세요(입력 안할시 현재 시간)
※ 참고 - 예: 2021-03-18 18:15:43
입력:  


It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...0
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...1
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...2
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...3
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...4
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...5
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...6
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...7
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...8
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...9
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...10
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...11
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...12
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...13
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...14
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...15
It failed JSONDecodeError
TypeError
서버와의 연결 불안정 재시도중...16
It failed JSONDecodeErro

## 데이터 CSV로 저장

In [75]:
# 폴더가 없을 경우
def createFolder(directory):
    import os
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [76]:
for coin, info in coins_week_info.items():
    createFolder('./'+interval)
    coin_code = coin_codes[coin_codes.korean_name == coin]
    code = coin_code[coin_code.market.str.contains('KRW')]['market'].iloc[0]
    info.to_csv('{}/{}.csv'.format(interval, code))

## (선택)엑셀에 코인가격 등락에 따른 색 적용

In [5]:
# 등락 셀 색상 적용(오를 경우 빨간색, 내려갈 경우 파란색)
def draw_color_cell(row):
    global pre_val
    color = [''] * len(pre_val)
    for i, col in enumerate(pre_val):
        if row[i] > col:
            color[i] = '#ff9090'
        elif row[i] < col:
            color[i] = '#0055FF'
        else: color[i] = 'white'
            
    pre_val = [row['open'], row['high'], row['low'], row['close'], row['volume'], row['value']]
        
    background=[f'background-color:{color[0]}',f'background-color:{color[1]}',f'background-color:{color[2]}',f'background-color:{color[3]}',f'background-color:{color[4]}',f'background-color:{color[5]}']
    return background

## 데이터 Exel로 저장

In [8]:
# 등락에 대한 배경 색 설정 후 Exel로 저장
for coin, info in coins_week_info.items():
    pre_val = [pre_open, pre_high, pre_low, pre_close, pre_volume, pre_value] = info.iloc[0]
    info.style.apply(draw_color_cell, axis=1).to_excel('{}.xlsx'.format(coin), engine='openpyxl')#0055FF 파란색